In [1]:
!pip install transformer
!pip install sentencepiece
!pip install pyvi --quiet
# !pip install --upgrade PyYAML transformers huggingface_hub

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings('ignore')  # Tắt toàn bộ cảnh báo Python

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"      # Tắt cảnh báo TensorFlow (nếu có)
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"  # Tắt warning advisory của transformers

# Nếu dùng logging, tắt logging warning
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
logging.getLogger("matplotlib").setLevel(logging.ERROR)
logging.getLogger("numba").setLevel(logging.ERROR)
logging.getLogger("datasets").setLevel(logging.ERROR)
from torch.utils.data import Dataset
import pandas as pd
from transformers import T5EncoderModel, AutoTokenizer
import pytorch_lightning as pl
from torch.utils.data import Dataset

E0000 00:00:1762460107.449579      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762460107.499512      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AsymmetricLossOptimized(nn.Module):
    ''' Optimized Asymmetric Loss for multi-label/multi-class with imbalance '''

    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=False):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

        self.targets = self.anti_targets = self.xs_pos = self.xs_neg = self.asymmetric_w = self.loss = None

    def forward(self, x, y):
        # x: logits [B, num_aspects, 4], y: one-hot [B, num_aspects, 4]
        self.targets = y
        self.anti_targets = 1 - y

        # Probabilities
        self.xs_pos = torch.sigmoid(x)
        self.xs_neg = 1.0 - self.xs_pos

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            self.xs_neg.add_(self.clip).clamp_(max=1)

        # Basic CE
        self.loss = self.targets * torch.log(self.xs_pos.clamp(min=self.eps))
        self.loss.add_(self.anti_targets * torch.log(self.xs_neg.clamp(min=self.eps)))

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            self.xs_pos = self.xs_pos * self.targets
            self.xs_neg = self.xs_neg * self.anti_targets
            self.asymmetric_w = torch.pow(1 - self.xs_pos - self.xs_neg,
                                          self.gamma_pos * self.targets + self.gamma_neg * self.anti_targets)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            self.loss *= self.asymmetric_w

        return -self.loss.sum() / x.size(0)   # lấy mean loss cho mỗi batch


In [4]:
def smooth_labels(labels, smoothing=0.1, num_classes=4):
    """
    labels: Tensor [B, num_aspects, num_classes], one-hot.
    smoothing: float, ví dụ 0.1.
    num_classes: int, số class mỗi aspect.
    """
    with torch.no_grad():
        labels = labels * (1 - smoothing) + (smoothing / num_classes)
    return labels

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import AutoModel
import math

# 1. Neighborhood Attention
class NeighborhoodAttention(nn.Module):
    def __init__(self, hidden_size: int, num_heads: int = 8, window_size: int = 7):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.window_size = window_size
        self.head_dim = hidden_size // num_heads
        self.qkv = nn.Linear(hidden_size, hidden_size * 3, bias=False)
        self.proj = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        B, L, C = x.shape
        qkv = self.qkv(x).reshape(B, L, 3, self.num_heads, self.head_dim)
        q, k, v = qkv.permute(2, 0, 3, 1, 4)
        attn_output = []
        for i in range(L):
            start = max(0, i - self.window_size // 2)
            end = min(L, i + self.window_size // 2 + 1)
            q_i = q[:, :, i:i+1, :]
            k_window = k[:, :, start:end, :]
            v_window = v[:, :, start:end, :]
            scores = torch.matmul(q_i, k_window.transpose(-2, -1)) / math.sqrt(self.head_dim)
            attn_weights = F.softmax(scores, dim=-1)
            out_i = torch.matmul(attn_weights, v_window)
            attn_output.append(out_i)
        attn_output = torch.cat(attn_output, dim=2)
        attn_output = attn_output.transpose(1, 2).reshape(B, L, C)
        return self.proj(attn_output)

# 2. StateConvBlock (conv + gating)
class StateConvBlock(nn.Module):
    def __init__(self, hidden_size: int):
        super().__init__()
        inner_dim = hidden_size * 2
        self.in_proj = nn.Linear(hidden_size, inner_dim * 2, bias=False)
        self.conv1d = nn.Conv1d(inner_dim, inner_dim, kernel_size=3, padding=1, groups=inner_dim)
        self.out_proj = nn.Linear(inner_dim, hidden_size, bias=False)
        self.norm = nn.LayerNorm(hidden_size)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_and_res = self.in_proj(x)
        x_inner, res = x_and_res.split(x_and_res.shape[-1] // 2, dim=-1)
        x_inner = self.conv1d(x_inner.transpose(1, 2)).transpose(1, 2)
        x_inner = F.silu(x_inner)
        y = x_inner * F.silu(res)
        output = self.out_proj(y)
        return x + self.norm(output)

# 3. Position Biased Attention + Aspect pooling
class PositionBiasedAttention(nn.Module):
    """
    Multi-head self-attention with positional bias for aspect pooling.
    """
    def __init__(self, hidden_size: int, num_aspects: int):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_aspects = num_aspects
        self.attn = nn.MultiheadAttention(hidden_size, num_heads=8, batch_first=True)
        self.aspect_queries = nn.Parameter(torch.randn(num_aspects, hidden_size))
    def _create_position_bias(self, seq_len: int, device: torch.device) -> torch.Tensor:
        bias = torch.zeros(seq_len, seq_len, device=device)
        for i in range(seq_len):
            for j in range(seq_len):
                distance = abs(i - j)
                if distance > 5:
                    bias[i, j] = -10.0
        return bias
    def forward(self, x: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        B, L, D = x.shape
        pos_bias = self._create_position_bias(L, x.device)
        attn_out, _ = self.attn(x, x, x, attn_mask=pos_bias)
        # Aspect pooling: for mỗi aspect, query toàn bộ sequence
        aspect_features = []
        for b in range(B):
            seq_len = attention_mask[b].sum().item()
            keys = attn_out[b, :seq_len].unsqueeze(0)  # [1, seq_len, D]
            queries = self.aspect_queries.unsqueeze(0)  # [1, num_aspects, D]
            # Sử dụng scaled dot-product attention (không dùng Multihead nữa)
            attn_score = torch.matmul(queries, keys.transpose(-2, -1)) / math.sqrt(D)  # [1, num_aspects, seq_len]
            attn_weight = F.softmax(attn_score, dim=-1)
            pooled = torch.matmul(attn_weight, keys)  # [1, num_aspects, D]
            aspect_features.append(pooled.squeeze(0))
        return torch.stack(aspect_features)  # [B, num_aspects, D]

# 4. AdvancedCNN branch
class AdvancedCNN(nn.Module):
    def __init__(self, hidden_size: int, num_filters: int = 128):
        super().__init__()
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(hidden_size, num_filters, kernel_size=3, dilation=1, padding=1),
            nn.Conv1d(hidden_size, num_filters, kernel_size=3, dilation=2, padding=2),
            nn.Conv1d(hidden_size, num_filters, kernel_size=3, dilation=4, padding=4),
        ])
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fusion = nn.Linear(num_filters * len(self.conv_layers), hidden_size)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.transpose(1, 2)
        conv_outputs = []
        for conv in self.conv_layers:
            conv_out = F.relu(conv(x))
            pooled = self.pool(conv_out).squeeze(-1)
            conv_outputs.append(pooled)
        fused = torch.cat(conv_outputs, dim=-1)
        return self.fusion(fused)

# 5. Main ABSA Model
class AdvancedABSAModel(pl.LightningModule):
    def __init__(self, 
                 num_aspects=12, 
                 learning_rate=2e-5, 
                 encoder_name='VietAI/vit5-large',
                 use_neighborhood_attn=True,
                 use_state_conv=True,
                 use_position_attn=True,
                 use_advanced_cnn=True):
        super().__init__()
        self.save_hyperparameters()
        self.num_aspects = num_aspects
        self.encoder = T5EncoderModel.from_pretrained(encoder_name)
        hidden_size = self.encoder.config.d_model

        self.use_neighborhood_attn = use_neighborhood_attn
        self.use_state_conv = use_state_conv
        self.use_position_attn = use_position_attn
        self.use_advanced_cnn = use_advanced_cnn

        if self.use_neighborhood_attn:
            self.neighborhood_attn = NeighborhoodAttention(hidden_size, window_size=7)
        if self.use_state_conv:
            self.state_conv_blocks = nn.ModuleList([StateConvBlock(hidden_size) for _ in range(2)])
        if self.use_position_attn:
            self.position_biased_attn = PositionBiasedAttention(hidden_size, num_aspects)
        if self.use_advanced_cnn:
            self.advanced_cnn = AdvancedCNN(hidden_size)
        self.fusion = nn.Linear(hidden_size * 2, hidden_size)
        self.classifiers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_size, hidden_size // 2),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_size // 2, 4)
            ) for _ in range(num_aspects)
        ])
        self.criterion = AsymmetricLossOptimized(
            gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=False
        )


    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        encoder_output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        x = encoder_output.last_hidden_state  # [B, L, D]
    
        if self.use_neighborhood_attn:
            x_block = self.neighborhood_attn(x, attention_mask)
        else:
            x_block = x
    
        if self.use_state_conv:
            for block in self.state_conv_blocks:
                x_block = block(x_block)
    
        if self.use_position_attn:
            aspect_features = self.position_biased_attn(x_block, attention_mask)  # [B, num_aspects, D]
        else:
            # Global pooling thay thế
            aspect_features = x_block.mean(dim=1, keepdim=True).expand(-1, self.num_aspects, -1)
    
        if self.use_advanced_cnn:
            cnn_features = self.advanced_cnn(x_block)  # [B, D]
        else:
            cnn_features = x_block.mean(dim=1)  # [B, D]
    
        cnn_features = cnn_features.unsqueeze(1).expand(-1, self.num_aspects, -1)  # [B, num_aspects, D]
        fused_features = torch.cat([aspect_features, cnn_features], dim=-1)
        fused_features = self.fusion(fused_features)
        logits = []
        for i, classifier in enumerate(self.classifiers):
            aspect_logits = classifier(fused_features[:, i, :])  # [B, 4]
            logits.append(aspect_logits)
        logits = torch.stack(logits, dim=1)  # [B, num_aspects, 4]
        return logits


    def training_step(self, batch, batch_idx):
        logits = self(batch['input_ids'], batch['attention_mask'])  # [B, num_aspects, 4]
        labels = batch['labels']   # [B, num_aspects] or [B, num_aspects, 4]
        if labels.dim() == 2:
            labels = F.one_hot(labels, num_classes=4).float()
        # --- LABEL SMOOTHING ở đây ---
        labels = smooth_labels(labels, smoothing=0.1, num_classes=4)
        main_loss = self.criterion(logits, labels)
        preds = torch.argmax(logits, dim=-1)
        targets = torch.argmax(labels, dim=-1)
        acc = (preds == targets).float().mean()
        self.log('train_loss', main_loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return main_loss

    def validation_step(self, batch, batch_idx):
        logits = self(batch['input_ids'], batch['attention_mask'])
        labels = batch['labels']
        if labels.dim() == 2:
            labels = F.one_hot(labels, num_classes=4).float()
        preds = torch.argmax(logits, dim=-1)
        targets = torch.argmax(labels, dim=-1)
        acc = (preds == targets).float().mean()
        self.log('val_acc', acc, prog_bar=True)
        return acc

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate, weight_decay=0.01)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-7)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "interval": "epoch"}
        }

In [6]:
class ABSADataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=256):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.aspect_columns = [col for col in self.data.columns if col != 'Review']
        self.num_aspects = len(self.aspect_columns)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = str(row['Review'])
        encoding = self.tokenizer(
            text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt'
        )
        labels = []
        for aspect in self.aspect_columns:
            sentiment = int(row[aspect])
            one_hot = [0, 0, 0, 0]
            one_hot[sentiment] = 1
            labels.append(one_hot)
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(labels, dtype=torch.float),
            'review_text': text
        }

class ABSADataModule(pl.LightningDataModule):
    def __init__(self, train_csv, val_csv, test_csv, tokenizer_name="VietAI/vit5-large", batch_size=16, max_length=256):
        super().__init__()
        self.train_csv = train_csv
        self.val_csv = val_csv
        self.test_csv = test_csv
        self.batch_size = batch_size
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = ABSADataset(self.train_csv, self.tokenizer, self.max_length)
            self.val_dataset = ABSADataset(self.val_csv, self.tokenizer, self.max_length)
        if stage == 'test' or stage is None:
            self.test_dataset = ABSADataset(self.test_csv, self.tokenizer, self.max_length)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, generator=torch.Generator(device='cuda'))
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)


In [7]:
import os
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

# Bạn cần import các thành phần model, tokenizer từ transformers nếu có dùng BERT, Roberta,...
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import logging

# Nếu có cần dùng các tiện ích khác
import random

# Nếu có dùng cuda kiểm tra
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')


In [8]:
import os
import numpy as np
import torch
from tqdm.notebook import tqdm

idx_to_sentiment = {0: 'None', 1: 'positive', 2: 'negative', 3: 'neutral'}

def generate_prediction_file(model, dataloader, device, output_path, aspect_columns):
    model.eval()
    all_preds = []
    texts = dataloader.dataset.data['Review'].tolist()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="🔍 Predicting", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            preds = torch.argmax(outputs, dim=-1).cpu().numpy()
            all_preds.append(preds)
    all_preds = np.vstack(all_preds)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, line in enumerate(texts):
            f.write(f"#{i+1}\n{line}\n")
            aspects = [
                f"{{{aspect_columns[j]}, {idx_to_sentiment[pred]}}}"
                for j, pred in enumerate(all_preds[i]) if pred != 0
            ]
            f.write(", ".join(aspects) + "\n\n" if aspects else "\n\n")
    return output_path

def generate_gold_file(dataloader, output_path, aspect_columns):
    all_labels = []
    texts = dataloader.dataset.data['Review'].tolist()
    for batch in tqdm(dataloader, desc="📄 Loading gold", leave=False):
        labels = batch['labels'].cpu().numpy()  # [B, A, 4]
        all_labels.append(labels)
    all_labels = np.vstack(all_labels)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, line in enumerate(texts):
            f.write(f"#{i+1}\n{line}\n")
            aspects = [
                f"{{{aspect_columns[j]}, {idx_to_sentiment[np.argmax(all_labels[i][j])]}}}"
                for j in range(len(aspect_columns)) if np.argmax(all_labels[i][j]) != 0
            ]
            f.write(", ".join(aspects) + "\n\n" if aspects else "\n\n")
    return output_path



In [9]:
import re

def get_labels_from_filename(filename):
    """Đọc nhãn từ file prediction/gold theo format:
    #1
    review_text
    {ASPECT1, positive}, {ASPECT2, neutral}
    """
    labels = []
    with open(filename, 'r', encoding='utf-8') as file:
        datasets = file.read()
        count = 0
        for line in datasets.split('\n'):
            if line != '':
                if count == 0:      # ID
                    count += 1
                elif count == 1:    # Review text
                    count += 1
                elif count == 2:    # Aspect-sentiment pairs (hoặc rỗng)
                    labels.append(line.strip())
                    count = 0
            elif count == 2:        # Empty line sau review text (không có aspect)
                labels.append("")
                count = 0
    return labels

def clean_label(label):
    # Chỉ giữ chữ, số, dấu # và &
    label = re.sub('[^A-Za-z0-9#&]', '', label)
    label = re.sub('\\s+', ' ', label)
    return label.strip()

def convert_labels_to_dict(labels):
    """Chuyển 1 dòng nhãn thành dict {aspect: polarity}"""
    dict_labels = []
    for label in labels:
        if not label:
            dict_labels.append({})
            continue
        label_line = label.split('},')
        _dict = {}
        for objectLabel in label_line:
            if not objectLabel.strip():
                continue
            parts = objectLabel.split(',')
            if len(parts) < 2:
                continue
            aspect = clean_label(parts[0]).strip()
            polarity = clean_label(parts[1]).strip()
            _dict[aspect] = polarity
        dict_labels.append(_dict)
    return dict_labels

def get_common_attributeEntities(dict_labels):
    AttributeEntities = []
    for _dict in dict_labels:
        for key in _dict:
            if key not in AttributeEntities:
                AttributeEntities.append(key)
    AttributeEntities = sorted(AttributeEntities)
    return AttributeEntities

def get_aspects(dict_labels):
    aspects = []
    for _dict in dict_labels:
        for key in _dict:
            aspects.append(key)
    return aspects

def count_aspects(labels, Common_AttributeEntities):
    aspects = get_aspects(labels)
    num_aspects = [0] * len(Common_AttributeEntities)
    for aspect in aspects:
        num_aspects[Common_AttributeEntities.index(aspect)] += 1
    return num_aspects

def evaluation_labels(gold_labels, answer_labels, Common_AttributeEntities):
    num_aspect_gold = count_aspects(gold_labels, Common_AttributeEntities)
    num_aspect_answer = count_aspects(answer_labels, Common_AttributeEntities)
    correct_answer_aspects = [0] * len(Common_AttributeEntities)
    correct_answer_labels = [0] * len(Common_AttributeEntities)

    for i, _dict in enumerate(answer_labels):
        for key in _dict:
            if key in gold_labels[i].keys():
                correct_answer_aspects[Common_AttributeEntities.index(key)] += 1
                if answer_labels[i][key].lower() == gold_labels[i][key].lower():
                    correct_answer_labels[Common_AttributeEntities.index(key)] += 1
    print('Correct Aspect Detection per Aspect:', correct_answer_aspects)
    print('Correct Aspect+Sentiment Detection:', correct_answer_labels)
    print('---------------------------------------------------')
    print('Aspect Detection Metrics:')
    infor_evaluation(correct_answer_aspects, num_aspect_answer, num_aspect_gold, Common_AttributeEntities)
    print('---------------------------------------------------')
    print('Aspect+Sentiment Detection Metrics:')
    infor_evaluation(correct_answer_labels, num_aspect_answer, num_aspect_gold, Common_AttributeEntities)

def infor_evaluation(correct_answer, num_aspect_answer, num_aspect_gold, Common_AttributeEntities):
    for idx, aspect in enumerate(Common_AttributeEntities):
        if correct_answer[idx] == 0:
            p = r = f = 0.0
        else:
            p = correct_answer[idx] * 100 / num_aspect_answer[idx] if num_aspect_answer[idx] > 0 else 0.0
            r = correct_answer[idx] * 100 / num_aspect_gold[idx] if num_aspect_gold[idx] > 0 else 0.0
            f = 2 * p * r / (p + r) if (p + r) > 0 else 0.0
        print(f"{aspect}: Precision={p:.2f}, Recall={r:.2f}, F1={f:.2f}")
    p = sum(correct_answer) * 100 / sum(num_aspect_answer) if sum(num_aspect_answer) > 0 else 0.0
    r = sum(correct_answer) * 100 / sum(num_aspect_gold) if sum(num_aspect_gold) > 0 else 0.0
    f = 2 * p * r / (p + r) if (p + r) > 0 else 0.0
    print('-------------------------------------------------------------')
    print(f'Mean Precision: {round(p,2)}')
    print(f'Mean Recall: {round(r,2)}')
    print(f'Mean F1: {round(f,2)}')
    print('-------------------------------------------------------------')

def evaluation_system(gold_labels, answer_labels):
    gold_dicts = convert_labels_to_dict(gold_labels)
    answer_dicts = convert_labels_to_dict(answer_labels)
    AttributeEntities = get_common_attributeEntities(gold_dicts)
    evaluation_labels(gold_dicts, answer_dicts, AttributeEntities)

def evaluation_system_by_file(file_gold, file_predict):
    gold_labels = get_labels_from_filename(file_gold)
    answer_labels = get_labels_from_filename(file_predict)
    evaluation_system(gold_labels, answer_labels)

In [10]:
ablation_settings = [
    # # 1. Tắt Neighborhood Attention
    # {"use_neighborhood_attn": False, "use_state_conv": True, "use_position_attn": True, "use_advanced_cnn": True},

    # # 2. Tắt StateConvBlock
    # {"use_neighborhood_attn": True, "use_state_conv": False, "use_position_attn": True, "use_advanced_cnn": True},

    # # 3. Tắt Position Biased Attention
    # {"use_neighborhood_attn": True, "use_state_conv": True, "use_position_attn": False, "use_advanced_cnn": True},

    # # 4. Tắt AdvancedCNN
    # {"use_neighborhood_attn": True, "use_state_conv": True, "use_position_attn": True, "use_advanced_cnn": False},

    # # 5. Tắt hết (chỉ dùng backbone encoder, ablation "full vanilla")
    # {"use_neighborhood_attn": False, "use_state_conv": False, "use_position_attn": False, "use_advanced_cnn": False},

    # # 6. Tắt Neighborhood Attention + StateConvBlock (giữ 2 module cuối)
    # {"use_neighborhood_attn": False, "use_state_conv": False, "use_position_attn": True, "use_advanced_cnn": True},

    # # 7. Tắt StateConvBlock + Position Attention
    # {"use_neighborhood_attn": True, "use_state_conv": False, "use_position_attn": False, "use_advanced_cnn": True},

    # # 8. Tắt Position Attention + AdvancedCNN
    # {"use_neighborhood_attn": True, "use_state_conv": True, "use_position_attn": False, "use_advanced_cnn": False},

    # # 9. Tắt Neighborhood Attention + AdvancedCNN
    # {"use_neighborhood_attn": False, "use_state_conv": True, "use_position_attn": True, "use_advanced_cnn": False},

    # # 10. Tắt Neighborhood Attention + Position Attention
    # {"use_neighborhood_attn": False, "use_state_conv": True, "use_position_attn": False, "use_advanced_cnn": True},

    # 11. Tắt StateConvBlock + AdvancedCNN
    # {"use_neighborhood_attn": True, "use_state_conv": False, "use_position_attn": True, "use_advanced_cnn": False},

    # 12. Tắt Neighborhood Attention + StateConvBlock + Position Attention
    # {"use_neighborhood_attn": False, "use_state_conv": False, "use_position_attn": False, "use_advanced_cnn": True},

    # 13. Tắt Neighborhood Attention + StateConvBlock + AdvancedCNN
    # {"use_neighborhood_attn": False, "use_state_conv": False, "use_position_attn": True, "use_advanced_cnn": False},

    # 14. Tắt Neighborhood Attention + Position Attention + AdvancedCNN
    # {"use_neighborhood_attn": False, "use_state_conv": True, "use_position_attn": False, "use_advanced_cnn": False},

    # 15. Tắt StateConvBlock + Position Attention + AdvancedCNN
    # {"use_neighborhood_attn": True, "use_state_conv": False, "use_position_attn": False, "use_advanced_cnn": False},
    # 16. Có đầy đủ các thành phần 
    {"use_neighborhood_attn": True, "use_state_conv": True, "use_position_attn": True, "use_advanced_cnn": True}
]

model_type="VietAI/vit5-base"
learning_rate=2e-5
batch_size=16
max_epochs=20
gpus=1
train_csv="/kaggle/input/uit-absa-resturant-dataset/Train.csv"
val_csv="/kaggle/input/uit-absa-resturant-dataset/Dev.csv"
test_csv="/kaggle/input/uit-absa-resturant-dataset/Test.csv"
max_length=128
checkpoint_dir="/kaggle/working/"
log_dir="/kaggle/working/"
mixed_precision=True
accumulate_grad_batches=1

print("🚀 Starting Advanced ABSA Training...")
print(f"📊 Configuration:")
print(f"   - Model: {model_type}")
print(f"   - Learning Rate: {learning_rate}")
print(f"   - Batch Size: {batch_size}")
print(f"   - Max Epochs: {max_epochs}")
print(f"   - GPU: {gpus}")

import pytorch_lightning as pl

class EvaluateOnTestSetCallback(pl.Callback):
    def __init__(self, test_dataloader, aspect_columns, gold_file_path, predict_file_path, device):
        self.test_dataloader = test_dataloader
        self.aspect_columns = aspect_columns
        self.gold_file_path = gold_file_path
        self.predict_file_path = predict_file_path
        self.device = device

    def on_validation_epoch_end(self, trainer, pl_module):
        print("-" * 10)
        epoch = trainer.current_epoch + 1  # Đếm từ 1
        print(f"\nEvaluating on Test Set - Epoch {epoch}:")
        print("-" * 10)
        # Gọi hàm generate prediction/gold
        generate_prediction_file(pl_module, self.test_dataloader, self.device, self.predict_file_path, self.aspect_columns)
        generate_gold_file(self.test_dataloader, self.gold_file_path, self.aspect_columns)
        # Evaluate
        evaluation_system_by_file(self.gold_file_path, self.predict_file_path)



# Setup data module
dm = ABSADataModule(
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=test_csv,
    batch_size=batch_size,
    max_length=max_length,
    tokenizer_name=model_type
)

# Get number of aspects from dataset
dm.setup()
num_aspects = dm.train_dataset.num_aspects
aspect_columns = dm.test_dataloader().dataset.aspect_columns
print(f"   - Number of Aspects: {num_aspects}")

count = 0
for setting in ablation_settings:
    print(f"Running ablation: {count+1}", setting)
    model = AdvancedABSAModel(num_aspects=num_aspects , learning_rate=learning_rate, encoder_name='VietAI/vit5-large', **setting)
    early_stopping = EarlyStopping(monitor="val_acc", mode="max", patience=2, verbose=True)
         
    evaluate_callback = EvaluateOnTestSetCallback(
        test_dataloader=dm.test_dataloader(),
        aspect_columns=aspect_columns,          
        gold_file_path="predictions/test_predictions.txt",
        predict_file_path="predictions/gold_labels.txt",
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    checkpoint_callback = ModelCheckpoint(
        dirpath=checkpoint_dir,  # Thư mục lưu model (đã định nghĩa là "/kaggle/working/")
        filename=f"ablation_{count+1}_best_model", # Tên file, thêm count để phân biệt
        monitor="val_acc",       # Theo dõi chỉ số 'val_acc'
        mode="max",              # Lưu model khi chỉ số này lớn nhất
        save_top_k=1,            # Chỉ lưu 1 model tốt nhất
        verbose=True
    )

    # Setup trainer
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        # THÊM checkpoint_callback VÀO ĐÂY:
        callbacks=[early_stopping, evaluate_callback, checkpoint_callback],
        precision=16 if mixed_precision else 32,
        gradient_clip_val=1.0,
        accumulate_grad_batches=accumulate_grad_batches,
        check_val_every_n_epoch=1,
        log_every_n_steps=449,
        logger=False,
        enable_checkpointing=True
    )
    
    # Train model
    print("\n🔥 Starting Training...")
    trainer.fit(model, dm)
    
    print(f"\n✅ Training Complete!")
    


🚀 Starting Advanced ABSA Training...
📊 Configuration:
   - Model: VietAI/vit5-base
   - Learning Rate: 2e-05
   - Batch Size: 16
   - Max Epochs: 20
   - GPU: 1


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

----------

Evaluating on Test Set - Epoch 6:
----------


🔍 Predicting:   0%|          | 0/122 [00:00<?, ?it/s]

📄 Loading gold:   0%|          | 0/122 [00:00<?, ?it/s]

Correct Aspect Detection per Aspect: [201, 42, 177, 95, 102, 510, 358, 94, 208, 102, 96, 278]
Correct Aspect+Sentiment Detection: [181, 35, 137, 76, 91, 465, 285, 75, 171, 92, 83, 249]
---------------------------------------------------
Aspect Detection Metrics:
AMBIENCE#GENERAL: Precision=93.49, Recall=88.55, F1=90.95
DRINKS#PRICES: Precision=89.36, Recall=89.36, F1=89.36
DRINKS#QUALITY: Precision=93.65, Recall=87.19, F1=90.31
DRINKS#STYLE&OPTIONS: Precision=82.61, Recall=73.64, F1=77.87
FOOD#PRICES: Precision=77.86, Recall=91.07, F1=83.95
FOOD#QUALITY: Precision=90.59, Recall=92.06, F1=91.32
FOOD#STYLE&OPTIONS: Precision=87.96, Recall=81.92, F1=84.83
LOCATION#GENERAL: Precision=93.07, Recall=90.38, F1=91.71
RESTAURANT#GENERAL: Precision=72.22, Recall=82.87, F1=77.18
RESTAURANT#MISCELLANEOUS: Precision=85.00, Recall=70.34, F1=76.98
RESTAURANT#PRICES: Precision=94.12, Recall=82.05, F1=87.67
SERVICE#GENERAL: Precision=96.86, Recall=91.75, F1=94.24
---------------------------------------